### 用爬蟲抓取已發行股數資料

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

### 建立抓取資料函數

> 這邊會需要注意兩個地方

>> 1.用requests搜尋網址，BeautifulSoup找到資料所在的位置。

>> 2.因為是同一個網址內的搜尋欄位(輸入ticker)，所以一定要加timesleep

In [2]:
def get_data_df(url, stockNo):
    form_data = {
        'encodeURIComponent': 1,
        'step': 1,
        'firstin': 1,
        'off': 1,
        'queryName': 'co_id',
        'inpuType': 'co_id',
        'TYPEK': 'all',
        'isnew': False,
        'co_id': stockNo,
        'date1': 100,
        'date2': 110,
        'qryType': 1
    }
    
    r = requests.post(url, form_data)
    soup = BeautifulSoup(r.text, 'html.parser')
    tables = soup.find_all('table')
    
    ## 接下來就是從html裡面找資料

    table = tables[0]                  # 本網頁的第一個表格
    rows = table.find_all('tr')        # 從所有row裡面找資料
    
    # 用於儲存抓到的數據
    data = []
    
    for row in rows:
        th = row.find('th', class_='dColor nowrap')
        if th and ("已發行普通股數或TDR原股發行股數" in th.text):
            td = row.find('td', class_='lColor')
            if td:
                # 將結果存入列表中
                data.append([stockNo, td.text.strip()])
    
    # 將數據轉換為 DataFrame
    if data:
        df = pd.DataFrame(data, columns=['證券代碼', '已發行普通股數或TDR原股發行股數'])
    
    # 如果股票代碼有錯的話
    else: 
        print(f"股票代碼 {stockNo} 未找到相關數據。")
        df = pd.DataFrame()
    
    return df

### 用for迴圈抓取所需要的資料

In [ ]:
# 從 Excel 文件中讀取股票代碼列表
stock_list_df = pd.read_excel('證券代碼.xlsx')
stock_list = stock_list_df['Ticker'].tolist()

# 所有股票的數據並合併為一個 DataFrame
all_data = pd.DataFrame()

for stock in stock_list:

    df = get_data_df("https://mops.twse.com.tw/mops/web/t05st03", stock)
    
    if not df.empty:
        all_data = pd.concat([all_data, df], ignore_index=True)

    time.sleep(5)  # 增加 5 秒延遲，防止伺服器過載(一定要加，我被擋過)。

# 將合併後的 DataFrame 存儲到 Excel 文件中
if not all_data.empty:

    all_data.to_excel('已發行普通股數或TDR原股發行股數.xlsx', index=False)
    print("資料已存成 Excel 檔案: 已發行普通股數或TDR原股發行股數.xlsx")

else:
    print("未找到任何有效數據，Excel 檔案未創建。")
